In [1]:
import numpy as np
import math
from pathlib import Path

In [2]:
class Qitem:
    """Abstraction of the representation of a qbit as a "quantum"-item of the knapsack problem.
    
    Attributes
    ----------
    value : int
        Object value represented by the qbit.
    weight : int
        Object weight represented by the qbit.
    alpha : float, optional
        Alpha value of qbit representation (default is math.sqrt(1/2)).
    beta : float, optional
        Beta value of qbit representation (default is math.sqrt(1/2)).
    """
    
    def __init__(self, value=None, weight=None, alpha=math.sqrt(1/2), beta=math.sqrt(1/2)):
        self.alpha = alpha
        self.beta = beta
        self.value = value
        self.weight = weight
    
    def measure(self):
        """Measures the qbit value from comparison with a random number between [0,1).
        
        Returns
        -------
        measure : int
            0 or 1 depending on the qbit (alpha and beta) and the random number drawn.
        """
        return 1 if np.random.random_sample() < self.beta**2 else 0
    
    def update(self, matrix):
        """Update qbit alpha and beta values by applying matrix.
        
        Attributes
        ----------
        matrix :
            Two-dimensional matrix (qgate) to be applied to the qbit.
        """
        self.alpha = matrix[0][0] * self.alpha + matrix[0][1] * self.beta
        self.beta = matrix[1][0] * self.alpha + matrix[1][1] * self.beta

In [3]:
def matrix_move_gate(delta):
    """Generate a move-gate matrix to operate under a qitem with the given angle."""
    return [[math.cos(delta), -math.sin(delta)], [math.sin(delta), math.cos(delta)]]

In [4]:
def measure_individuals(Q):
    """Measure each qbit of the qitem population and return the result (list of 0s and 1s)."""
    return [q.measure() for q in Q]

In [5]:
def fit_solution(Q, solution):
    """Evaluate value and weight of the given solution.
    
    Attributes
    ----------
    Q : [Qitem]
        Qitem population.
    solution : [int]
        Solution from a population measurement.
        
    Returns
    -------
    value : int
        Total value of the evaluated solution.
    weight : int
        Total weight of the evaluated solution.
    """
    value = 0
    weight = 0

    for i, measure in enumerate(solution):
        value += measure*(Q[i].value)
        weight += measure*(Q[i].weight)
    
    return value, weight

In [6]:
def repair_solution(Q, solution, w_capacity, v_solution, w_solution):
    """Repairs the solution to make it a valid solution.
    If the sum of the item weights exceeds the limit, 
    it removes items at random until the constraint is satisfied.
    
    Attributes
    ----------
    Q : [Qitem]
        Qitem population.
    solution : [int]
        Solution from a population measurement.
    w_capacity : int
        Knapsack weight limit capacity.
    v_solution : int
        Total value of solution to be repaired.
    w_solution : int
        Total weight of solution to be repaired.
    
    Returns
    -------
    v_solution : int
        Total value of the repaired solution.
    w_solution : int
        Total weight of the repaired solution.
    """
    while w_solution > w_capacity:
        index = np.random.randint(0, len(solution)-1)
        if solution[index]:
            solution[index] = 0
            v_solution -= Q[index].value
            w_solution -= Q[index].weight
    
    return v_solution, w_solution

In [7]:
def fit_and_repair(Q, solution, w_capacity):
    """Evaluates (value and weight) and repairs a solution.
    
    Attributes
    ----------
    Q : [Qitem]
        Qitem population.
    solution : [int]
        Solution from a population measurement.
    w_capacity : int
        Knapsack weight limit capacity.
    
    Returns
    -------
    v_solution : int
        Total value of the evaluated and repaired solution.
    w_solution : int
        Total weight of the evaluated and repaired solution.
    """
    v_solution, w_solution = fit_solution(Q, solution)
    if w_solution > w_capacity:
        v_solution, w_solution = repair_solution(Q, solution, w_capacity, v_solution, w_solution)
    return v_solution, w_solution

In [8]:
def get_neighborhood(Q, population):
    """Measures qitem population to generate a population of neighboring solutions.
    
    Attributes
    ----------
    Q : [Qitem]
        Qitem population.
    population : int
        Number of neighboring populations to be generated.
    
    Returns
    -------
    [solution : [int]]
        List of neighboring solutions (list of different measurements of the population).
    """
    return [[q.measure() for q in Q] for p in range(population)]

In [9]:
def fit_and_repair_neighborhood(Q, neighborhood, w_capacity):
    """Evaluates (value and weight) and repairs all neighboring solutions.
    
    Attributes
    ----------
    Q : [Qitem]
        Qitem population
    neighborhood : [[int]]
        List of neighboring solutions (list of different measurements).
    w_capacity : int
        Knapsack weight limit capacity
    
    Returns
    -------
    [[solution : [int], value : int, weight : int]]
        List of repaired solution and corresponding evaluation (value and weight).
    """
    solutions = []
    for i, solution in enumerate(neighborhood):       
        solutions.append([solution, *fit_and_repair(Q, solution, w_capacity)])
    return solutions

In [10]:
def update_state(Q, delta, T, tabu_itt, solution_current, solution_comparison, is_best):
    """Update each qbit of the population by applying the matrix 
    according to the current solution and the comparison solution.
    
    Attributes
    ----------
    Q : [Qitem]
        Qitem population
    delta : float
        Angle used to build the move-gate matrix and operate under solution states.
    T : dict {int : int}
        Tabu list (implemented as a dictionary)
    tabu_itt : int
        Number of iterations an item should be kept on the tabu list
    solution_current : [int]
        Current solution from a population measurement.
    solution_comparison : [int]
        Solution to be compared with the current.
    is_best : bool
        True if the comparison solution was the best found from neighbors (worst otherwise).
    """
    for i, q in enumerate(Q):
        if T.setdefault(i, 0) == 0:
            continue
        diff = solution_comparison[i] - solution_current[i]
        if diff == 0:
            continue
        if not is_best: 
            diff *= -1
        if Q[i].alpha * Q[i].beta < 0:
            diff *= -1
        
        Q[i].update(matrix_move_gate(delta*diff))
        T[i] = tabu_itt

In [11]:
def qts(itt, delta, population, tabu_itt, file):
    """Performs the quantum-inspired tabu search algorithm.
    
    Attributes
    ----------
    itt : int
        Number of iterations for the algorithm to execute
    delta : float
        Angle used to build the move-gate matrix and operate under solution states.
    population : int
        Number of neighboring populations to be generated in each loop.
    tabu_itt : int
        Number of iterations an item should be kept on the tabu list
    file: Path
        Knapsack problem instance file
    
    Returns
    -------
    best_sol : [solution : [int], value : int, weight : int]
        Best solution found for the knapsack problem
    best_itt : int
        Iteration where the best solution was found
    """
    
    Q = []
    T = dict()
    w_capacity = 0
    n_items = 0
    optimum = 0
    solution_i = None
    
    best_sol = []
    best_itt = -1

    with open(file) as f:
        n_items = int(f.readline().split()[1])
        w_capacity = int(f.readline().split()[1])
        optimum = int(f.readline().split()[1])
        f.readline()
        for line in f:
            _, lineV, lineW, _ = list(map(int, line.split(',')))
            Q.append(Qitem(lineV, lineW))
    
    solution_i = measure_individuals(Q)

    value_i, weight_i = fit_and_repair(Q, solution_i, w_capacity)
    
    best_sol = [solution_i, value_i, weight_i]
    
    itt_count = 0
    itt_not_change = 0
    while itt_count < itt and itt_not_change < 200:
        itt_count += 1
        neighborhood_population = get_neighborhood(Q, population)
        neighborhood = fit_and_repair_neighborhood(Q, neighborhood_population, w_capacity)
        best_neighbor = max(neighborhood, key=lambda x: x[1])
        worst_neighbor = min(neighborhood, key=lambda x: x[1])
        
        found_better_solution = (
            best_neighbor[1] > best_sol[1] or 
            (best_neighbor[1] == best_sol[1] and best_neighbor[2] < best_sol[2])
        )
        
        if found_better_solution:
            best_sol = best_neighbor[:]
            best_itt = itt_count
            itt_not_change = 0
        else:
            itt_not_change +=1
            
        for key, value in list(T.items()):
            T[key] -= 1
            if T[key]==0:
                del T[key]
        
        update_state(Q, delta, T, tabu_itt, solution_i, best_neighbor[0], True)
        solution_i = measure_individuals(Q)
        
        update_state(Q, delta, T, tabu_itt, solution_i, worst_neighbor[0], False)
        solution_i = measure_individuals(Q)

    return best_sol, best_itt

In [12]:
knapsack_instance = Path('./data/knapPI_11_500_1000_1.csv')
# knapsack_instance = Path('./data/knapPI_12_500_1000_1.csv')
# knapsack_instance = Path('./data/knapPI_13_500_1000_1.csv')
# knapsack_instance = Path('./data/knapPI_1_5000_1000000_1.csv')

best_sol, best_itt = qts(1000, 0.01*math.pi, 10, 2, knapsack_instance)
print('Best Solution:', best_sol[1], 'on itt:', best_itt)

Best Solution: 3468 on itt: 11
